## Classificação dos dados
### Classificação supervisionada

In [40]:
dfa = df_arrumado

Nosso target é a largura do bico (bill length) dos pinguins, e, visto que é um target numérico, o agruparemos em 5 grupos:\
#### Grupos
1. (Muito pequeno) 0.0 até 0.2 \
dfa["bill_length_norm"] <= 0.2
2. (Pequeno) 0.2 até 0.4 \
(dfa["bill_length_norm"] > 0.2) & (dfa["bill_length_norm"] <= 0.4)\
E assim por diante
3. (Médio) 0.4 até 0.6
4. (Grande) 0.6 até 0.8
5. (Muito grande) 0.8 até 1

Na célula seguinte, organizei o valor da largura dos bicos (normalizado) em ordem crescente.

In [20]:
dfa.sort_values(by=['bill_length_norm'])

,bill_length_norm,bill_depth_norm,flipper_length_norm,body_mass_norm
136,0.0,0.285714,0.271186,0.097222
92,0.036364,0.357143,0.101695,0.055556
64,0.050909,0.702381,0.305085,0.25
86,0.069091,0.47619,0.220339,0.194444
13,0.083636,0.630952,0.20339,0.173611
...,...,...,...,...
326,0.836364,0.345238,0.983051,0.875
209,0.861818,0.797619,0.59322,0.361111
313,0.865455,0.464286,0.949153,0.805556
163,0.941818,0.559524,0.152542,0.277778


A seguir, aplicamos o código para classificar os bicos dos pinguins.

In [22]:
dfa["Largura do bico (categórico)"] = None

divisao = dfa["bill_length_norm"] <= 0.2
dfa.loc[divisao,"Largura do bico (categórico)"] = "Muito pequeno"
    
divisao = (dfa["bill_length_norm"] > 0.2) & (dfa["bill_length_norm"] <= 0.4) # Pequeno
dfa.loc[divisao,"Largura do bico (categórico)"] = "Pequeno"
    
divisao = (dfa["bill_length_norm"] > 0.4) & (dfa["bill_length_norm"] <= 0.6) # Médio
dfa.loc[divisao,"Largura do bico (categórico)"] = "Médio"
    
divisao = (dfa["bill_length_norm"] > 0.6) & (dfa["bill_length_norm"] <= 0.8) # Grande
dfa.loc[divisao,"Largura do bico (categórico)"] = "Grande"

divisao = (dfa["bill_length_norm"] > 0.8) & (dfa["bill_length_norm"] <= 1) # Muito grande
dfa.loc[divisao,"Largura do bico (categórico)"] = "Muito grande"

print(dfa)
print("Deu certo, apenas faltaria organizar a coluna em ordem crescente!")

     bill_length_norm  bill_depth_norm  flipper_length_norm  body_mass_norm  \
0            0.254545         0.666667             0.152542        0.291667   
1            0.269091         0.511905             0.237288        0.305556   
2            0.298182         0.583333             0.389831        0.152778   
3            0.167273         0.738095             0.355932        0.208333   
4            0.261818         0.892857             0.305085        0.263889   
..                ...              ...                  ...             ...   
328          0.549091         0.071429             0.711864        0.618056   
329          0.534545         0.142857             0.728814        0.597222   
330          0.665455         0.309524             0.847458        0.847222   
331          0.476364         0.202381             0.677966        0.694444   
332          0.647273         0.357143             0.694915            0.75   

    Largura do bico (categórico)  
0               

#### Classificação com k-NN

Agora, vamos começar a testar os três modelos passados em aula. O primeiro sendo o k-NN, ou nearest neighbors. 

In [48]:
from sklearn.model_selection import train_test_split

tamanho_teste = 0.1 
semente = 61454
#Definimos o tamanho do nosso teste e uma semente escolhida

features = ["body_mass_norm", "bill_depth_norm", "flipper_length_norm"] 
target = ["Largura do bico (categórico)"]
#Após isso, definimos as features que utilizaremos (neste caso todas as numéricas) nos nossos treinos e previsões
# para chegarmos no target.

In [42]:
indices = dfa.index
indices_treino, indices_teste = train_test_split(indices, test_size = tamanho_teste, random_state = semente)
#Definimos os indices que utilizaremos (do nosso data set). Com todos os indices do data set, escolhemos
# quais usaremos para treino e para teste.


dfa_treino = dfa.loc[indices_treino]
dfa_teste = dfa.loc[indices_teste]

x_treino = dfa_treino.reindex(features, axis=1).values
x_teste = dfa_teste.reindex(features, axis=1).values

y_treino = dfa_treino.reindex(target, axis=1).values.ravel()
y_teste = dfa_teste.reindex(target, axis=1).values.ravel()


In [25]:
from sklearn.neighbors import KNeighborsClassifier
import numpy

modelo_vizinhos = KNeighborsClassifier()
#Após termos importado o classificador, criamos o nosso modelo


modelo_vizinhos.fit(x_treino,y_treino)
# Então treinamos o modelo com nossas variáveis de treino, através do módulo .fit


previsao1 = modelo_vizinhos.predict(x_teste)
print(previsao1)

['Pequeno' 'Grande' 'Grande' 'Muito pequeno' 'Pequeno' 'Pequeno' 'Médio'
 'Pequeno' 'Grande' 'Grande' 'Médio' 'Grande' 'Pequeno' 'Pequeno'
 'Pequeno' 'Grande' 'Pequeno' 'Médio' 'Pequeno' 'Médio' 'Médio' 'Pequeno'
 'Grande' 'Pequeno' 'Médio' 'Pequeno' 'Muito pequeno' 'Grande' 'Pequeno'
 'Médio' 'Muito pequeno' 'Pequeno' 'Grande' 'Médio']


E, com isso, temos a nossa primeira previsão! Mas ela apenas utilizou dados de teste. A seguir, vamos utilizar os dados "verdadeiros". 

In [26]:
from sklearn import metrics

print("Precisão:", metrics.accuracy_score(y_teste, previsao1))

Precisão: 0.5


#### Classificação com árvore de decisão

In [27]:
from sklearn.tree import DecisionTreeClassifier

modelo_arvore = DecisionTreeClassifier(random_state = semente)

modelo_arvore.fit(x_treino, y_treino)

previsao2 = modelo_arvore.predict(x_teste)
print(previsao2)

['Grande' 'Grande' 'Grande' 'Pequeno' 'Médio' 'Pequeno' 'Médio' 'Pequeno'
 'Médio' 'Grande' 'Grande' 'Grande' 'Pequeno' 'Pequeno' 'Médio' 'Grande'
 'Muito pequeno' 'Médio' 'Pequeno' 'Pequeno' 'Médio' 'Grande' 'Grande'
 'Muito pequeno' 'Médio' 'Grande' 'Muito pequeno' 'Grande' 'Grande'
 'Grande' 'Muito pequeno' 'Pequeno' 'Muito grande' 'Médio']


In [28]:
print("Precisão:", metrics.accuracy_score(y_teste, previsao2))

Precisão: 0.4117647058823529


#### Classificação com florestas aleatórias

In [29]:
from sklearn.ensemble import RandomForestClassifier

modelo_floresta = RandomForestClassifier(random_state = semente)

modelo_floresta.fit(x_treino, y_treino)

previsao3 = modelo_arvore.predict(x_teste)
print(previsao3)

['Grande' 'Grande' 'Grande' 'Pequeno' 'Médio' 'Pequeno' 'Médio' 'Pequeno'
 'Médio' 'Grande' 'Grande' 'Grande' 'Pequeno' 'Pequeno' 'Médio' 'Grande'
 'Muito pequeno' 'Médio' 'Pequeno' 'Pequeno' 'Médio' 'Grande' 'Grande'
 'Muito pequeno' 'Médio' 'Grande' 'Muito pequeno' 'Grande' 'Grande'
 'Grande' 'Muito pequeno' 'Pequeno' 'Muito grande' 'Médio']


In [30]:
print("Precisão:", metrics.accuracy_score(y_teste, previsao3))

Precisão: 0.4117647058823529


As hipóteses para essas baixas precisões/accuracy são: o baixo número de parametros, a utilização APENAS de parametros numéricos e a não utilização de hiperparametros.

#### Tentativa de refinar os dados com hiperparâmetros

Assim como sugerido no final do bloco da última aula, tentei refinar os dados com hiperparametros utilizando o grid search. E, mesmo que tenha ajudado, creio que ainda seja necessário aumentar os parametros a serem utilizados.

In [31]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [200, 300, 400, 500, 600],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,6,8,10,12,14,16],
    'criterion' :['gini', 'entropy']
}

CV_rfc = GridSearchCV(estimator=modelo_floresta, param_grid=param_grid, cv= 5, n_jobs=8)
CV_rfc.fit(x_treino, y_treino)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=61454),
             n_jobs=8,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 6, 8, 10, 12, 14, 16],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [200, 300, 400, 500, 600]})

In [32]:
print(CV_rfc.best_params_)
best_params = CV_rfc.best_params_

{'criterion': 'gini', 'max_depth': 4, 'max_features': 'auto', 'n_estimators': 200}


In [33]:
best_params = {'criterion': 'entropy', 'max_depth': 14, 'max_features': 'sqrt', 'n_estimators': 600}

modelo_floresta = RandomForestClassifier(random_state=semente)
modelo_floresta.set_params(**best_params)


RandomForestClassifier(criterion='entropy', max_depth=14, max_features='sqrt',
                       n_estimators=600, random_state=61454)

In [47]:
modelo_floresta.fit(x_treino, y_treino)
previsao = modelo_floresta.predict(x_teste)
print(previsao)
print("Accuracy:",metrics.accuracy_score(y_teste, previsao))
print()
print("Melhorou bem pouco. Meu machine learning tão pequenininho....")

['Pequeno' 'Pequeno' 'Grande' 'Pequeno' 'Pequeno' 'Pequeno' 'Médio'
 'Pequeno' 'Grande' 'Grande' 'Médio' 'Grande' 'Pequeno' 'Pequeno' 'Grande'
 'Grande' 'Muito pequeno' 'Médio' 'Pequeno' 'Médio' 'Médio' 'Grande'
 'Grande' 'Pequeno' 'Médio' 'Grande' 'Pequeno' 'Grande' 'Grande' 'Grande'
 'Muito pequeno' 'Pequeno' 'Grande' 'Médio']
Accuracy: 0.5294117647058824

Melhorou bem pouco. Meu machine learning tão pequenininho....
